# Generar la reaccion a un texto

In [1]:
from string import punctuation
from collections import Counter

from textblob import TextBlob

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.corpus import wordnet as wn
from nltk import word_tokenize, pos_tag

## Paso 0: Definimos que es un interes

In [2]:
# Definimos una funcion para extraer las raices de las palabras
def lemmatize_stemming(text):
    stemmer = SnowballStemmer("english")
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

In [3]:
class Interest:
    def __init__(self, keyword, strenght=0.5, polarity=0.5):
        # Get synonyms
        self._keywords = []
        if wn.synsets(keyword):
            for word, tag in pos_tag(wn.synsets(keyword)[0].lemma_names()):
                if '_' in word:
                    continue
                    
                if tag == 'NNP':
                    self._keywords += [word.lower()]
                else:
                    self._keywords += [lemmatize_stemming(word).lower()]
        else:
            self._keywords = [lemmatize_stemming(keyword)]
            
        self._strenght = strenght
        self._polarity = polarity
    
    def get_list(self):
        return (self._keywords, self._strenght, self._polarity)

    def get_keywords(self):
        return self._keywords

    def get_polarity(self):
        return self._polarity

    def get_strenght(self):
        return self._strenght

## Paso 1: Preparamos uno interes y un texto al que relacionar

In [4]:
# A alguien que no le gustan los gatos
interest = Interest("cat", 0.8, 0.1)

# Un texto de alguien a quien le gustan los gatos
text = "Cats are just the best! Garfield is getting lasagna tonight"

In [5]:
# Intereses de alguien que ama los perros y odia los gatos
interests = [Interest("dog", 0.7, 0.9),
             Interest("cat", 0.8, 0.1)
            ]

# Textos con opiniones variadas sobre perros y gatos
texts = [
    "Man, last night a dog bite me, and it sucked, I hate dogs from now on",
    "Just got a new dog and its name is Kale! We are so excited!!",
    "Cats are just the best! Garfield is getting lasagna tonight",
    "Garfield the cat is a fat asshole"
]

## Paso 2: Extraemos las raices de las palabras del texto

In [6]:
# Definimos una funcion que nos dice si la palabra es un sustantivo
is_useful = lambda pos: pos[:2] in ['NN']#, 'VB']

raices=[]

# Por cada palabra comprobamos que sea un sustantivo, y si lo es, lo metemos en la lista de raices
for token, tag in pos_tag(word_tokenize(text)):
    if token[0] == '@':
        continue
    if not is_useful(tag):
        continue

    if token and token[0] not in gensim.parsing.preprocessing.STOPWORDS and len(token) >= 3:
        res = token if tag == 'NNP' else lemmatize_stemming(token)
        raices.append(res.lower())

# Ordenamos las raices por numero de ocurrencias
raices.sort(key=Counter(raices).get, reverse=True)

# Quitamos los duplicados
raices = list(dict.fromkeys(raices))

print('''
En el texto: 
{}
Existen las siguientes raices utiles:
{}
'''.format(text, raices))


En el texto: 
Cats are just the best! Garfield is getting lasagna tonight
Existen las siguientes raices utiles:
['cat', 'garfield', 'tonight']



Comprobamos que alguna de estas raices tenga que ver con nuestros intereses

In [7]:
matching_keywords = [k for k in interest.get_keywords() if k in raices]

if matching_keywords:
    print(f"Hay intereses presentes: {matching_keywords}")
else:
    print("No hay intereses comunes en el texto, la reaccion es [0.0, 0.0]")

Hay intereses presentes: ['cat']


## Paso 3: Calculamos el sentimiento del texto

In [8]:
blob = TextBlob(text)
text_sentiment = 0.5 + blob.sentiment.polarity/2

print(f"El sentimiento del texto es: {text_sentiment}")

El sentimiento del texto es: 1.0


## Paso 4: Calculamos como de correcto nos parece el texto

In [9]:
our_strenght = interest.get_strenght()
our_polarity = interest.get_polarity()

# Sentimos que el texto es correcto si se alinea con nuestra percepcion
if (our_polarity-0.5 < 0):
    percieved_text_rightfulness = 1-text_sentiment
else:
    percieved_text_rightfulness = text_sentiment

print(f"Percibimos que el texto es [{percieved_text_rightfulness}] correcto (0 es nada, 1 es completamente)")

Percibimos que el texto es [0.0] correcto (0 es nada, 1 es completamente)


## Paso 5: Calculamos nuestra reaccion al texto

In [11]:
# Definimos una funcion para calcular la radicalidad de una polaridad
calc_radicality = lambda p:abs(p-0.5)/0.5

our_radicality = calc_radicality(our_polarity)

# Calculamos la polaridad de nuestra reaccion
react_polarity = our_radicality * percieved_text_rightfulness + 0.5 * (1-our_radicality)

# Calculamos la fuerza de la reaccion
react_strenght = 0.5*calc_radicality(react_polarity) + 0.5 * our_strenght 

print('''
Sobre nuestro interes [{}] nuestra opinion es asi de radical:
{}

Nuestra reaccion es:
Texto: \t\t{}
Polaridad:\t{}
Intensidad:\t{}
'''.format(interest.get_keywords(), our_radicality, text, react_polarity, react_strenght))


Sobre nuestro interes [['cat']] nuestra opinion es asi de radical:
0.8

Nuestra reaccion es:
Texto: 		Cats are just the best! Garfield is getting lasagna tonight
Polaridad:	0.09999999999999998
Intensidad:	0.8

